In [1]:
from sfcn import SFCN

In [2]:
model = SFCN(
    input_dim=[160, 192, 160, 1], 
    output_dim=1,
    conv_num_filters=[32, 64, 128, 256, 256, 64], 
    conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
    conv_strides=[1, 1, 1, 1, 1, 1],
    conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
    maxpool_size=[2, 2, 2, 2, 2],
    batch_norm=False,
    softmax=False
    )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 160, 192, 160, 1) 0         
_________________________________________________________________
conv_0 (Conv3D)              (None, 160, 192, 160, 32) 64        
_________________________________________________________________
maxpool_0 (MaxPooling3D)     (None, 80, 96, 80, 32)    0         
_________________________________________________________________
activation_0 (Activation)    (None, 80, 96, 80, 32)    0         
_________________________________________________________________
conv_1 (Conv3D)              (None, 80, 96, 80, 64)    2112      
_________________________________________________________________
maxpool_1 (MaxPooling3D)     (None, 40, 48, 40, 64)    0         
_________________________________________________________________
activation_1 (Activation)    (None, 40, 48, 40, 64)    0     

In [3]:
model.compile(learning_rate=1e-3)

In [4]:
import numpy as np 

In [5]:
data_size = 200

In [9]:
X = np.random.randint(0, 3, size=(data_size, 160, 192, 160, 1)).astype('float')

In [2]:
y = np.sum(X, axis=(1,2,3,4))

NameError: name 'np' is not defined

In [11]:
model.train(X, y, 4, 100)

Epoch 1/100


ValueError: in user code:

    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Bram\miniconda3\envs\dl38\lib\site-packages\keras\engine\training.py:835 run_step

In [ ]:
x_test = np.random.randint(0, 2, size=(1, 64, 64, 64, 1)).astype('float')

In [ ]:
model.predict(x_test)

array([[0.04806068]], dtype=float32)

In [ ]:
y_test = np.mean(x_test, axis=(1,2,3,4))
print(y_test)

[0.49977493]
